In [67]:
import os
import sys
import pandas as pd
data_path = os.path.abspath('./data_all/data/diff/')
data_path

'd:\\Github\\django-shop\\handpose-mouse-simulation\\data_all\\data\\diff'

In [112]:
resList = []
df_list = []
label_list = []
for filename in os.listdir(data_path):
    if filename[-3:] =="csv":
        new_path = data_path + "\\"+ filename
        resList.append(new_path)
        label_list.append(filename[:-4])
        df = pd.read_csv(new_path , delimiter=',', header=0, skiprows=0,
                                   error_bad_lines=False)
        df_list.append(df)
        #df_list.extend(df)

In [27]:
label_list

['Chinese_seven_diff',
 'Click_diff',
 'Ok_pose_diff',
 'Relax_diff',
 'Rock_diff',
 'Up_down_diff']

In [102]:
class dataset():
    def __init__(self, data, lable):
        self.data = data
        self.lable = lable
        self.len = len(data)

In [113]:
#d1_train = dataset(df_list[0],label_list[0])
d1_train = dataset(df_list,label_list[0])
d1_train.lable
d1_train.data


[      -0.00033479928970336914  -0.0023109912872314453  6.41061706119217e-06  \
 0                    0.002057               -0.002193              0.000008   
 1                   -0.003012               -0.003290             -0.000017   
 2                   -0.000389               -0.002900              0.000008   
 3                   -0.002330               -0.000561             -0.000016   
 4                    0.000000               -0.003600              0.000009   
 ...                       ...                     ...                   ...   
 2602                 0.007303                0.002958             -0.000007   
 2603                 0.412453                0.260215             -0.000019   
 2604                -0.347082               -0.313046              0.000048   
 2605                 0.009684                0.001036              0.000039   
 2606                 0.000827               -0.003932             -0.000019   
 
       0.00011456012725830078  0.00150

In [56]:
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# import the libraries for classification modelling
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, mean_squared_error

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, transforms
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [58]:
device

device(type='cuda', index=0)

In [80]:
class CNNLSTMNet(nn.Module):
    def __init__(self, cnn_model, output_dim, hidden_dim,num_classes=10,seq_len=20 ,batch_size=1, num_lstm_layers = 1, bidirectional = False, device = 'cpu', freeze_layers=True, dropout=0, title="default"):
        super(CNNLSTMNet, self).__init__()
        # CNN
        self.device = device
        self.title = title # Model Title
        self.cnn_model = cnn_model # Torchvision CNN Model
        
        # Optionally Freeze CNN Layers
        if freeze_layers:
            for idxc, child in enumerate(self.cnn_model.children()):
                for param in child.parameters():
                    param.requires_grad = False
            self.cnn_model.fc.requires_grad = True
            
        # RNN
        self.seq_len = seq_len
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim # LSTM Hidden Dimension Size
        self.num_lstm_layers = num_lstm_layers 
        self.bidirectional = bidirectional # Sets LSTM to Uni or Bidirectional
        self.bidirectional_mult = 2 if self.bidirectional else 1 # Used for LSTM Weight Shape
        self.lstm = nn.LSTM(output_dim, hidden_dim, self.num_lstm_layers, bidirectional=self.bidirectional, dropout=dropout)
        self.hidden2class = nn.Linear(hidden_dim*self.bidirectional_mult, num_classes) # Fully Connected Output Layer
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(self.num_lstm_layers*self.bidirectional_mult, self.batch_size, self.hidden_dim).to(device),
                    torch.zeros(self.num_lstm_layers*self.bidirectional_mult, self.batch_size, self.hidden_dim).to(device))
                    
    def forward(self, x):
        x = x.view(self.seq_len*self.batch_size,x.shape[-3],x.shape[-2],-1)
        out = self.cnn_model(x)
        seq = out.view(self.batch_size, self.seq_len, -1).transpose_(0,1)
        self.hidden = self.init_hidden()
        # LSTM input shape = (seq_len, batch, input_size)
        out, self.hidden = self.lstm(seq.view(len(seq), self.batch_size, -1), self.hidden)
        #LSTM output shape = (seq_len, batch, hidden_dim * bidirectional)
        out = self.hidden2class(out[-1])
        return out

In [81]:
num_img_features = 1024 # CNN output dimensions
num_epochs = 10
sequence_len = d1_train.shape # LSTM sequence length
batch_size=3
hidden_dim = 128 # LSTM hidden dimension size
lstm_dropout = .1
lstm_depth = 1
freeze = False # True = Freeze entire CNN, False = Don't freeze any layers
pretrain = True # Use Imagenet Pretraining with CNN
lstm_depth_title = 'no_freeze_layers_num_lstm_layers_'+str(lstm_depth)
num_classes = len(label_list)+1
loss_fn = nn.CrossEntropyLoss()


In [87]:


title =\
'-dataset-'+str("test")+\
'-frozen-'+str(freeze)+\
'-num_img_features-'+ str(num_img_features) +\
'-num_epochs-'+str(num_epochs)+\
'-sequence_len-'+str(sequence_len)+\
'-batch_size-'+str(batch_size)+\
'-lstm_dim-'+str(hidden_dim)+\
'-lstm_depth-'+str(lstm_depth)+\
'-pretrain-'+str(pretrain)+\
'CNN-resnet18'

cnn_model = models.resnet18(pretrained=pretrain) #Choose different CNN if desired
num_ftrs = cnn_model.fc.in_features
cnn_model.fc = nn.Linear(num_ftrs, num_img_features) # Change CNN output layer to desired dimension
model = CNNLSTMNet(cnn_model, num_img_features, hidden_dim, num_classes, sequence_len, batch_size, num_lstm_layers=lstm_depth, bidirectional=False, device=device, freeze_layers=freeze, dropout=lstm_dropout)
optimizer = optim.Adam(model.parameters(), lr = 1e-4)
model_save_path= 'saved_models/'+title+'.pth'
# Use Below to load train history and train over a saved model
# model.load_state_dict(torch.load(model_save_path))
print(title)


-dataset-test-frozen-False-num_img_features-1024-num_epochs-10-sequence_len-379-batch_size-3-lstm_dim-128-lstm_depth-1-pretrain-TrueCNN-resnet18


In [115]:
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torch
import torch.optim as optim
#from plot_model_stats import *
import os
import matplotlib.pyplot as plt
import time
import torchnet
from torchnet.meter import ConfusionMeter
import torch.nn.functional as F


def my_collate(batch):
    # Filters DataLoader items after they are retrieved
    data = [item[0] for item in batch if item != None]
    target = [item[1] for item in batch if item != None]
    if len(data) == 0: return [None, None]
    data = torch.stack(data)
    target = torch.LongTensor(target)
    return [data, target]


def train_model(model, loss_fn, batch_size, dataset, optimizer, model_title='None', device='cpu', root_dir='', num_epochs = 2, testset=None):
    def write_results(title, write_string):
        with open('model_results/'+title + '_results.txt', 'a+') as f:
            print(write_string)
            f.write(write_string + '\n')

    # Shuffling is needed in case dataset is not shuffled by default.
    train_ratio=.9
    train_len = int(len(dataset)*train_ratio)
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, len(dataset)-train_len]) 
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=my_collate
                                               )
    # We don't need to bach the validation set but let's do it anyway.
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=my_collate
                                             )  

    # GPU enabling.
    model = model.to(device)
    loss_fn = loss_fn.to(device)

    # Training loop. Please make sure you understand every single line of code below.
    # Go back to some of the previous steps in this lab if necessary.
    start_time = time.time()
    plot_path=os.path.join(root_dir,'model_results/'+model_title+'.png')
    print("Trainset Length: ", len(train_loader))
    print("Valset Length: ", len(val_loader))
    print("Epoch Count: ", num_epochs)
    print("Plot Path: ",plot_path)
    
    train_accuracies=[]; val_accuracies=[]; train_losses=[]; val_losses=[]
    for epoch in range(0, num_epochs):
        correct = 0.0
        cum_loss = 0.0

        # Make a pass over the training data.
        model.train()
        num_trained = 1.0
        confusion_matrix = torch.zeros(len(label_list), len(label_list))
        for (i, (inputs, labels)) in enumerate(train_loader):
            try:
                inputs = inputs.to(device)
                labels = labels.to(device)#[0]

                # Forward pass. (Prediction stage)
                scores = model(inputs)
                loss = loss_fn(scores, labels)

                # Zero the gradients in the network.
                optimizer.zero_grad()

                # Backward pass. (Gradient computation stage)
                loss.backward()

                # Parameter updates (SGD step) -- if done with torch.optim!
                optimizer.step()

                scores = F.softmax(scores)
                max_scores, max_labels = scores.max(1)
                correct_eval = (max_labels == labels).sum().item()
                correct+= correct_eval
                num_trained+=batch_size
                cum_loss += loss.item()
                _, preds = torch.max(scores, 1)
                for t, p in zip(labels.view(-1), preds.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1


                # Logging the current results on training.
                if (i + 1) % 100 == 0:
                    #print(dataset.labels)
                    write_string =('Train-epoch %d. Iteration %05d, Avg-Loss: %.4f, Accuracy: %.4f' %
                          (epoch, num_trained + 1, cum_loss / (num_trained), correct / (num_trained)))
                    write_string += '\n'+str(confusion_matrix)
                    confusion_avg = (confusion_matrix.diag()/confusion_matrix.sum(1))
                    write_string += '\n'+ str(confusion_avg)
                    write_results(model_title, write_string)
                    #print("Time Elapsed Minutes: ", (time.time() - start_time) /60)
                    correct = 0.0
                    cum_loss = 0.0
                    num_trained = 1.0
                    confusion_matrix = torch.zeros(len(label_list), len(label_list))
                    model_save_path= 'saved_models/'+model_title+'.pth'

                    torch.save(model.state_dict(), model_save_path)
            except Exception as e:
                
                print(e)
        train_accuracies.append(correct / num_trained)
        train_losses.append(cum_loss / num_trained)
        write_string = "Time Elapsed Minutes: "+ str((time.time() - start_time) /60)
        write_results(model_title, write_string)

        # Make a pass over the validation data.
        model.eval()
        num_trained = 1.0
        correct = 0.0
        cum_loss = 0.0

        confusion_matrix = torch.zeros(len(label_list), len(label_list))
        print('Validating...')
        #print(dataset.labels)
        for (i, (inputs, labels)) in enumerate(val_loader):
            try:
  
                inputs = inputs.to(device)
                labels = labels.to(device)#[0]

                # Forward pass. (Prediction stage)
                scores = model(inputs)
                latent_scores.append([scores.item(),labels.item()])
                loss = loss_fn(scores, labels)
                scores = F.softmax(scores)
                # Count how many correct in this batch.
                max_scores, max_labels = scores.max(1)
                correct_eval = (max_labels == labels).sum().item()
                correct+= correct_eval
                num_trained+=batch_size
                cum_loss += loss.item()
                _, preds = torch.max(scores, 1)
                for t, p in zip(labels.view(-1), preds.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1
                if i % 100 == 0:
                    print(confusion_matrix)
            except Exception as e:
                print(e)
        val_accuracies.append(correct / num_trained)
        val_losses.append(cum_loss / num_trained)
        write_string =('validation-epoch %d. Iteration %05d, Avg-Loss: %.4f, Accuracy: %.4f' %
              (epoch, num_trained + 1, cum_loss / num_trained, correct / num_trained ))
        write_string += '\n'+str(confusion_matrix)
        confusion_avg = (confusion_matrix.diag()/confusion_matrix.sum(1))
        write_string += '\n'+ str(confusion_avg)
        write_results(model_title, write_string)
        # plot_model_stats(train_accuracies, val_accuracies, train_losses, val_losses, plot_path, latent_scores=latent_scores)
        if testset != None:
            try:
                from test_model import test_model
                test_model(model,testset,dataset, batch_size, joint_run=joint_run)
            except Exception as e:
                print(e)



In [117]:
 (d1_train.data).all()

AttributeError: 'list' object has no attribute 'all'

In [116]:
train_model(model, loss_fn, batch_size, d1_train.data, optimizer, title, device, num_epochs=num_epochs)

Trainset Length:  2
Valset Length:  1
Epoch Count:  10
Plot Path:  model_results/-dataset-test-frozen-False-num_img_features-1024-num_epochs-10-sequence_len-379-batch_size-3-lstm_dim-128-lstm_depth-1-pretrain-TrueCNN-resnet18.png


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().